In [1]:
import os, re
import pandas as pd
import numpy as np
from analytics_utils.util.qubole import run_hive
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.style.use('seaborn-whitegrid')
%matplotlib inline
from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss, roc_curve

In [2]:
def getData(model_id, omni_model_id, pid, start_date, end_date):
    
    s_date = start_date.replace('-', '') 
    e_date = end_date.replace('-', '')
    
    query1 = """
    DROP TABLE IF EXISTS tmp_seed_{pid}_{s_date}_{e_date};
    CREATE TABLE IF NOT EXISTS tmp_seed_{pid}_{s_date}_{e_date} AS
    SELECT DISTINCT hhid
    FROM core_shared.transaction
    WHERE provider_id = {pid}
    AND txn_type = 'P'
    AND total_amount > 0
    AND txn_dt >= date '{start_date}'
    AND txn_dt < date '{end_date}'
    ;

    DROP TABLE IF EXISTS tmp_back_{pid}_{s_date}_{e_date};
    CREATE TABLE IF NOT EXISTS tmp_back_{pid}_{s_date}_{e_date} AS
    SELECT u.hhid as hhid, CASE WHEN s.hhid IS NULL THEN 0 ELSE 1 END AS response
    FROM dsmodeling.extract_hhids u
    LEFT JOIN tmp_seed_{pid}_{s_date}_{e_date} s on u.hhid = s.hhid
    ;

    """.format(
        pid = pid,
        start_date = start_date,
        end_date = end_date,
        s_date = s_date,
        e_date = e_date
    )
    
    run_hive(query1, token='zarFf9MKQiandxqBeD751rw5qikvnMyjVwhDrGyJqfm8o1KdLdo4R36btaNfmXAG', return_data=False)      

    query2 = """
    SELECT * FROM tmp_back_{pid}_{s_date}_{e_date}
    ;
    """.format(
        pid = pid,
        s_date = s_date,
        e_date = e_date
    )
    
    response_df = run_hive(query2, token='zarFf9MKQiandxqBeD751rw5qikvnMyjVwhDrGyJqfm8o1KdLdo4R36btaNfmXAG', return_data=True)      
    response_df.columns = ['hhid', 'response']

    query3 = """
    SELECT hhid, score FROM dsmodeling.ensemble_score_{model_id}_0;
    """.format(
        model_id = model_id
    )
    score_df = run_hive(query3, token='zarFf9MKQiandxqBeD751rw5qikvnMyjVwhDrGyJqfm8o1KdLdo4R36btaNfmXAG', return_data=True)      
    score_df.columns = ['hhid', 'score']

    query4 = """
    SELECT hhid, score FROM dsmodeling.ensemble_score_{omni_model_id}_0;
    """.format(
        omni_model_id = omni_model_id
    )
    score_omni_df = run_hive(query4, token='zarFf9MKQiandxqBeD751rw5qikvnMyjVwhDrGyJqfm8o1KdLdo4R36btaNfmXAG', return_data=True)      
    score_omni_df.columns = ['hhid', 'score']

    
    back_df = response_df.merge(score_df, how = 'inner', on = 'hhid')
    back_omni_df = response_df.merge(score_omni_df, how = 'inner', on = 'hhid')
    
    return back_df, back_omni_df

In [3]:
def getTop(top_val, back_df, back_omni_df):
    back_top_df = back_df.nlargest(top_val, 'score')
    back_omni_top_df = back_omni_df.nlargest(top_val, 'score')
    return back_top_df, back_omni_top_df

In [4]:
### ROC AUC and Precision Recall AUC ###
def plot_model_metrics_2(df_1, df_2, penetration_index, note, campaign_note, brand):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4.5))

    # ROC AUC 
    from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss, roc_curve
    from sklearn.calibration import calibration_curve
    import bisect
    import pylab as pl
    import sys
    with PdfPages(note[1:]+'_'+campaign_note+'.pdf') as pdf:   
        from matplotlib.gridspec import GridSpec
        gs = GridSpec(9, 1)
        plt.style.use('seaborn-whitegrid')
        roc_auc_1 = roc_auc_score(df_1['response'], df_1['score'])
        roc_auc_2 = roc_auc_score(df_2['response'], df_2['score'])
        fpr_1, tpr_1, _1 = roc_curve(df_1['response'], df_1['score'])
        fpr_2, tpr_2, _2 = roc_curve(df_2['response'], df_2['score'])
        ax1.plot(fpr_1, tpr_1, 'b-', alpha = 0.5, label='without omni (area = %0.3f)' % roc_auc_1)
        ax1.plot(fpr_2, tpr_2, 'r', alpha = 0.5, label='with omni (area = %0.3f)' % roc_auc_2)
        ax1.plot([0, 1], [0, 1], 'k--', alpha = 0.5)
        ax1.axis(xmin = 0.0, xmax = 1.0, ymin = 0.0, ymax = 1.05)
        ax1.set_xlabel('False Positive Rate (1 - Specifity)')
        ax1.set_ylabel('True Positive Rate (Sensitivity)')
        ax1.set_title('ROC Curve ' + note)
        ax1.legend(loc="lower right")

        #PRECISION (BTR)
        segments = 10
        top_segments = 1
        segment_percentile = [i for i in range(10, 110, segments)]

        '''model1'''
        df_sorted_1 = df_1.sort_values('score', ascending = False)
        df_sorted_1.reset_index(drop = 1, inplace = True)

        segment_size = np.round(len(df_sorted_1)/segments)
        segment_precision_1 = []
        for i in (np.array(range(segments)) + 1)*segment_size:
            segment_precision_1.append(round((df_sorted_1.loc[:i, 'response'].mean())/penetration_index, 3))
        cumulative_precision_1 = segment_precision_1[-1]
        
        '''model2'''
        df_sorted_2 = df_2.sort_values('score', ascending = False)
        df_sorted_2.reset_index(drop = 1, inplace = True)

        segment_size = np.round(len(df_sorted_2)/segments)
        segment_precision_2 = []
        for i in (np.array(range(segments)) + 1)*segment_size:
            segment_precision_2.append(round((df_sorted_2.loc[:i, 'response'].mean())/penetration_index, 3))
        cumulative_precision_2 = segment_precision_2[-1]

        segment_precision_frac = [round(segment_precision_2[i]/segment_precision_1[i],4) for i in range(len(segment_precision_1))]

        print "The segment precision for the base model:" + str(segment_precision_1)
        print "The segment precision for the model with omni data:" + str(segment_precision_2)

        for i in range(1,3):
            ax2.text((i-1)*10, segment_precision_2[i-1]+0.05, str((segment_precision_frac[i-1]-1)*100)+'%', color='black')
            x1, y1, x2, y2 = [i*10, i*10], [0, segment_precision_2[i-1]], [0, i*10], [segment_precision_2[i-1], segment_precision_2[i-1]]
            ax2.plot(x1, y1, 'k--', alpha = 0.5)
            ax2.plot(x2, y2, 'k--', alpha = 0.5)
            plt.plot(segment_percentile[i-1], segment_precision_2[i-1], marker='o', markersize=3, color="red")

        ax2.plot(segment_percentile, segment_precision_1, 'b-', alpha = 0.5, label = 'without omni')
        ax2.plot(segment_percentile, segment_precision_2, 'r', alpha = 0.5, label = 'with omni')
        ax2.axis(xmin = 0, xmax = 100, ymin = 0.00, ymax = max(max(segment_precision_1, segment_precision_2))+0.1)
        ax2.set_xlabel('Segment Percentile')
        ax2.set_ylabel('BTRi')
        ax2.set_title('Lift Table'+note)
        ax2.legend(loc="lower middle")
        
    
        #SCALE INCREASE
        scale_segments = 100
        scale_segment_size = np.round(len(df_sorted_2)/scale_segments)
        scale_segment_precision_2 = sys.maxint
        depth = 7
        
        while (segment_precision_1[0] < scale_segment_precision_2):
            scale_segment_precision_2 = round((df_sorted_2.loc[:depth*scale_segment_size, 'response'].mean())/penetration_index, 3)
            depth += 0.1

        txt = '''
        The BTRi for the base model at 10% segment : {precision_segment_1}
        The BTRi for the model with omni data at 10% segment : {precision_segment_2} ({BTRi_pct_1}% increase)
        The cumulative BTRi for the base model: {cumulative_precision_1}
        The cumulative BTRi for the model with omni data: {cumulative_precision_2} ({BTRi_pct}% increase)
        The scale of direct mail models (at comparable BTRi of segment 1) is changed by {scale_pct}%
        '''.format(
        precision_segment_1 = round(segment_precision_1[0],2),
        precision_segment_2 = round(segment_precision_2[0],2),
        BTRi_pct_1 = round((segment_precision_frac[0]-1)*100,2),
        cumulative_precision_1=cumulative_precision_1,
        cumulative_precision_2=cumulative_precision_2,
        BTRi_pct=round((cumulative_precision_2/cumulative_precision_1-1)*100,2),
        scale_pct = round(depth-0.1-10,1)*100/10
        )
        
        plt.text(0,-0.2,txt, transform=fig.transFigure, size=10)    
        pdf.savefig(bbox_inches="tight", pad_inches=0.5)
        plt.close()


In [6]:
start_date='2019-01-01'
end_date='2019-03-01'
campaign_note = 'real'
brand_list = ["J.Jill Group","Coldwater Direct","Northstyle","Pyramid Collection","K Jordan","Drapers & Damons"]
pid_list = [707, 4633, 64, 62, 1633, 560]
model_id_list = [16711, 16710, 16718, 16722, 16726, 16724]
omni_model_id_list = [16847, 16848, 16850, 16855, 16854, 16868]
top_list = [5000000, 10000000, 5000000, 40000000, 5000000, 500000]
for i in range(len(brand_list)):
    brand = brand_list[i]
    pid = pid_list[i]
    model_id = model_id_list[i]
    omni_model_id = omni_model_id_list[i]
    top = top_list[i]
    back_df, back_omni_df = getData(model_id, omni_model_id, pid, start_date, end_date)
    back_df = back_df.dropna()
    back_omni_df = back_omni_df.dropna()
    penetration_index = back_df['response'].mean()
    penetration_omni_index  = back_omni_df['response'].mean()
    print brand
    back_top_df, back_omni_top_df = getTop(top, back_df, back_omni_df)
    plot_model_metrics_2(back_top_df, back_omni_top_df, penetration_index, '_'+brand+'_'+str(top), campaign_note, brand)

K Jordan
The segment precision for the base model:[104.287, 54.411, 36.899, 27.939, 22.484, 18.842, 16.203, 14.221, 12.687, 11.433]
The segment precision for the model with omni data:[104.501, 54.386, 36.913, 27.98, 22.531, 18.873, 16.243, 14.253, 12.707, 11.459]
